In [1]:
%load_ext autoreload
%autoreload 2

import sys
sys.path.append('/Users/ira/Downloads/diplom/REALEC_Inspector')

import sqlite3
from tqdm import tqdm_notebook as tqdm
import os
import numpy as np
from utils.main import main
import pandas as pd
import collections
import statistics
from sklearn.compose import ColumnTransformer
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.tokenize import word_tokenize
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import FunctionTransformer
from sklearn.pipeline import FeatureUnion, Pipeline
from sklearn.svm import LinearSVC
from sklearn.metrics import classification_report
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import StratifiedKFold

import json
with open('result.json') as data_file:
    data = json.load(data_file)

SEED = 23
BASE_DIR = os.path.dirname(os.path.abspath(''))
PATH_DB = os.path.join(BASE_DIR, 'data', 'realec.db')
JSON_FILE = os.path.join(BASE_DIR, 'data', 'files_with_json.txt')

import warnings
warnings.filterwarnings("ignore")

In [2]:
def connect(dbfile):
    conn = sqlite3.connect(dbfile)
    c = conn.cursor()
    return c, conn

In [3]:
# xx = {'density': 0.43564356435643564, 'LS': 0.14772727272727273, 'VSI': 0.11764705882352941, 'VSII': 0.34299717028501764, 'VSIII': 0.23529411764705882, 'LFP_first_procent': 0.5643564356435643, 'LFP_second_procent': 0.06930693069306931, 'LFP_third_procent': 0.054455445544554455, 'LFP_none': 0.31188118811881194, 'NDW': 103, 'TTR': 0.5099009900990099, 'CTTR': 5.1244415295814445, 'RTTR': 7.247054710722006, 'LogTTR': 0.8731151577940734, 'Uber': 287.17064749454755, 'D': 0.8129560915759496, 'LV': 0.5099009900990099, 'VVI': 0.8823529411764706, 'SVVI': 13.235294117647058, 'CVVI': 2.5724787771376323, 'VVII': 0.17045454545454544, 'NV': 0.1188118811881188, 'AdjV': 0.17045454545454544, 'AdvV': 0.09090909090909091, 'ModV': 0.26136363636363635, 'der_suff3': 0.0, 'der_suff4': 0.1590909090909091, 'der_suff5': 0.022727272727272728, 'der_suff6': 0.0, 'MCI': 5.25, 'freq_finite_forms': 0.47058823529411764, 'freq_aux': 0.4117647058823529, 'infinitive_tokens': 4, 'gerund_tokens': 0, 'pres_sg_tokens': 9, 'pres_pl_tokens': 0, 'parts': 3, 'pasts': 1, 'link_Sentence connectors(Furthermore)': 0, 'link_Sentence connectors(In addition)': 0, 'link_Sentence connectors(Moreover)': 0, 'link_Sentence connectors': 0, 'link_Phrases linkers(In addition)': 0, 'link_Phrases linkers': 0, 'link_Addition': 0, 'link_subordinators(Although)': 0, 'link_subordinators(Even though)': 0, 'link_subordinators': 0, 'link_Sentence connectors(However)': 0, 'link_Sentence connectors(Nevertheless)': 0, 'link_Phrase linkers(despite)': 0, 'link_Phrase linkers(In spite of)': 0, 'link_Phrase linkers': 0, 'link_Adversativity': 0, 'link_subordinators(Because)': 0, 'link_subordinators(since)': 0, 'link_Sentence connectors(Therefore)': 0, 'link_Sentence connectors(As a result)': 0, 'link_Sentence connectors(consequently)': 0, 'link_Sentence connectors(Hence)': 0, 'link_Sentence connectors(Thus)': 0, 'link_Phrase linkers(Because of)': 0, 'link_Phrase linkers(Due to)': 0, 'link_Phrase linkers(As a result of)': 0, 'link_Cause and effect': 0, 'link_Sentence connectors(In other words)': 0, 'link_Sentence connectors(That is)': 0, 'link_Sentence connectors(i.e.)': 4, 'link_Clarification': 4, 'link_Subordinators(while)': 0, 'link_Subordinators(whereas)': 0, 'link_Subordinators': 0, 'link_Sentence connectors(In contrast)': 0, 'link_Sentence connectors(On the other hand)': 0, 'link_Sentence connectors(Conversely)': 0, 'link_Phrase linkers(Unlike)': 0, 'link_Contrast': 0, 'link_Sentence connectors(For example)': 0, 'link_Sentence connectors(For instance)': 0, 'link_Illustration': 0, 'link_Sentence connectors(On the contrary)': 0, 'link_Sentence connectors(As a matter of fact)': 0, 'link_Sentence connectors(In fact)': 0, 'link_Intensification': 0, 'link_all': 4, 'num_4grams': 1, '4grams_Topic elaboration/clarification(and to be a)': 0, '4grams_Topic elaboration/clarification(are more and more)': 0, '4grams_Topic elaboration/clarification(as well as the)': 0, '4grams_Topic elaboration/clarification(but there are still)': 0, '4grams_Topic elaboration/clarification(can be divided into)': 0, '4grams_Topic elaboration/clarification(how to deal with)': 0, '4grams_Topic elaboration/clarification(if you don’t know)': 0, '4grams_Topic elaboration/clarification(if you want to)': 0, '4grams_Topic elaboration/clarification(in order to make)': 0, '4grams_Topic elaboration/clarification(is a kind of)': 0, '4grams_Topic elaboration/clarification(is based on the)': 0, '4grams_Topic elaboration/clarification(is more important than)': 0, '4grams_Topic elaboration/clarification(is totally different from)': 0, '4grams_Topic elaboration/clarification(it is a good)': 0, '4grams_Topic elaboration/clarification(it is a very)': 0, '4grams_Topic elaboration/clarification(it is also a)': 0, '4grams_Topic elaboration/clarification(it is because the)': 0, '4grams_Topic elaboration/clarification(it is not a)': 0, '4grams_Topic elaboration/clarification(on the other hand)': 0, '4grams_Topic elaboration/clarification(there will be a)': 0, '4grams_Topic elaboration/clarification(to cope with the)': 0, '4grams_Topic elaboration/clarification(want to be a)': 0, '4grams_Topic elaboration/clarification': 0, '4grams_Identification/focus(my point of view)': 0, '4grams_Identification/focus(the best way to)': 0, '4grams_Identification/focus(a very important role)': 0, '4grams_Identification/focus(as far as the)': 0, '4grams_Identification/focus(as I have mentioned)': 0, '4grams_Identification/focus(him or her to)': 0, '4grams_Identification/focus(is one of my)': 0, '4grams_Identification/focus(is one of the)': 0, '4grams_Identification/focus(is the most important)': 0, '4grams_Identification/focus(is very important for)': 0, '4grams_Identification/focus(it is very important)': 0, '4grams_Identification/focus(one of the most)': 0, '4grams_Identification/focus(the most important thing)': 0, '4grams_Identification/focus(we can say that)': 0, '4grams_Identification/focus(we can see that)': 0, '4grams_Identification/focus(we can see the)': 0, '4grams_Identification/focus': 0, '4grams_Topic introduction(I am going to)': 0, '4grams_Topic introduction(I would like to)': 0, '4grams_Topic introduction(if there is a)': 0, '4grams_Topic introduction': 0, '4grams_Discourse organizers': 0, '4grams_Quantifying(a great deal of)': 0, '4grams_Quantifying(a great number of)': 0, '4grams_Quantifying(a large amount of)': 0, '4grams_Quantifying(a lot of people)': 0, '4grams_Quantifying(a lot of problem)': 0, '4grams_Quantifying(a lot of problems)': 0, '4grams_Quantifying(a lot of time)': 0, '4grams_Quantifying(all of them are)': 0, '4grams_Quantifying(and a lot of)': 0, '4grams_Quantifying(bring a lot of)': 0, '4grams_Quantifying(has a lot of)': 0, '4grams_Quantifying(more and more people)': 0, '4grams_Quantifying(most of the people)': 0, '4grams_Quantifying(most of them are)': 0, '4grams_Quantifying(some of them are)': 0, '4grams_Quantifying(that it is more)': 0, '4grams_Quantifying(the rest of the)': 0, '4grams_Quantifying(the rest of the world)': 0, '4grams_Quantifying(there are a lot of)': 0, '4grams_Quantifying(there are many people)': 0, '4grams_Quantifying(there are quite a)': 0, '4grams_Quantifying(there are so many)': 0, '4grams_Quantifying(there are still some)': 0, '4grams_Quantifying(there are too many)': 0, '4grams_Quantifying(with a lot of)': 0, '4grams_Quantifying': 0, '4grams_Time/place/text deixis(all over the world)': 0, '4grams_Time/place/text deixis(at the beginning of the)': 0, '4grams_Time/place/text deixis(at the same time)': 0, '4grams_Time/place/text deixis(for a long time)': 0, '4grams_Time/place/text deixis(in the following paragraphs)': 0, '4grams_Time/place/text deixis(the end of the)': 0, '4grams_Time/place/text deixis': 0, '4grams_Framing(because they are not)': 0, '4grams_Framing(in such a way)': 0, '4grams_Framing(in the process of)': 0, '4grams_Framing(on the basis of)': 0, '4grams_Framing(the main reason is)': 0, '4grams_Framing(the quality of the)': 0, '4grams_Framing(the reason is that)': 0, '4grams_Framing(the relationship between the)': 0, '4grams_Framing(the result of the)': 0, '4grams_Framing(with the development of)': 0, '4grams_Framing(as a result of)': 0, '4grams_Framing(as the result of)': 0, '4grams_Framing(the result of this)': 0, '4grams_Framing': 0, '4grams_Referential': 0, '4grams_Epistemic(as a matter of)': 0, '4grams_Epistemic(as we all know)': 0, '4grams_Epistemic(become more and more)': 0, '4grams_Epistemic(I think it is)': 0, '4grams_Epistemic(I think that this)': 0, '4grams_Epistemic(I think the most)': 0, '4grams_Epistemic(I think this is)': 0, '4grams_Epistemic(it is believed that)': 0, '4grams_Epistemic(it is obvious that)': 0, '4grams_Epistemic(it is true that)': 0, '4grams_Epistemic(some people think that)': 0, '4grams_Epistemic': 0, '4grams_Attitudinal/modality(are not allowed to)': 0, '4grams_Attitudinal/modality(I hope I can)': 0, '4grams_Attitudinal/modality(is very important to)': 0, '4grams_Attitudinal/modality(it is difficult to)': 0, '4grams_Attitudinal/modality(it is very difficult)': 0, '4grams_Attitudinal/modality(it is hard to)': 0, '4grams_Attitudinal/modality(it is not easy)': 0, '4grams_Attitudinal/modality(necessary for us to)': 0, '4grams_Attitudinal/modality(should learn how to)': 0, '4grams_Attitudinal/modality(will not be able to)': 0, '4grams_Attitudinal/modality': 0, '4grams_Stance': 0, '4grams_all': 0, 'av_depth': 3.5384615384615383, 'max_depth': 5, 'min_depth': 2, 'acl': 4, 'rel_cl': 2, 'advcl': 0, 'count_sent': 13, 'count_tokens': 202, 'tokens_before_root': 3.1538461538461537, 'mean_len_sent': 15.538461538461538}

In [4]:
# new = {}
# for x in xx:
#     new[x] = []
# new['name'] = []
# new['text'] = []
# new['target'] = []

In [5]:
# import json

# import io
# try:
#     to_unicode = unicode
# except NameError:
#     to_unicode = str

# # Write JSON file
# with io.open('result.json', 'w', encoding='utf8') as outfile:
#     str_ = json.dumps(new,
#                       indent=4, sort_keys=True,
#                       separators=(',', ': '), ensure_ascii=False)
#     outfile.write(to_unicode(str_))



In [6]:
with open(JSON_FILE, 'r') as rf:
    clean_essays = rf.read().split('\n')

# Find features

In [7]:
c, conn = connect(PATH_DB)
c.execute("SELECT TEXT, MARK, NAME FROM MAIN")
result = c.fetchall()
for essay in tqdm(result):
    text = essay[0]
    mark = essay[1]
    name = essay[2]
    if name in clean_essays:
        try:
            result = main(text)
            for key in result:
                data[key].append(result[key])
            data['name'].append(name)
            data['text'].append(text)
            data['target'].append(mark)
        except:
            continue

In [8]:
df = pd.DataFrame(data=data)
df.head()
df.to_csv('dataset.csv', index=False)

In [9]:
df = pd.read_csv('dataset.csv')
df.head(3)

,4grams_Attitudinal/modality,4grams_Attitudinal/modality(I hope I can),4grams_Attitudinal/modality(are not allowed to),4grams_Attitudinal/modality(is very important to),4grams_Attitudinal/modality(it is difficult to),4grams_Attitudinal/modality(it is hard to),4grams_Attitudinal/modality(it is not easy),4grams_Attitudinal/modality(it is very difficult),4grams_Attitudinal/modality(necessary for us to),4grams_Attitudinal/modality(should learn how to),...,name,num_4grams,parts,pasts,pres_pl_tokens,pres_sg_tokens,rel_cl,target,text,tokens_before_root
0,0,0,0,0,0,0,0,0,0,0,...,./data/exam/exam2017/OBy_146_1,1,2,0,0,4,0,60,This two line graphs illustrates monthly avera...,4.388889
1,0,0,0,0,0,0,0,0,0,0,...,./data/exam/exam2017/EGe_15_1,1,1,4,0,5,3,65,"﻿We have two graphics, which show us the popul...",6.285714
2,1,0,0,1,0,0,0,0,0,0,...,./data/exam/exam2017/DOv_2_2,5,2,9,0,5,0,60,More and more young people are stunding on the...,4.562500


In [10]:
df.shape

(3399, 217)

# Classification

In [11]:
feature_columns = list(data.keys())
feature_columns.remove('target')
feature_columns.remove('name')
feature_columns.remove('text')
feature_columns = feature_columns

In [12]:
# feature_columns

In [13]:
def tokenizer(text):
    return word_tokenize(text)

In [14]:
# tokenizer('I am Ira')

In [15]:
# tfidf = TfidfVectorizer(lowercase=True, tokenizer=tokenizer)
X = df[feature_columns]
targets = []
for t in df['target']:
    if int(t) >= 70:
        targets.append('best')
    elif int(t) < 60:
        targets.append('worst')
    else:
        targets.append('average')
y = targets

counter=collections.Counter(y)
print(counter)

Counter({'average': 2068, 'worst': 950, 'best': 381})


In [16]:
# feature_columns

In [17]:
get_numeric_data = FunctionTransformer(lambda x: x[feature_columns], validate=False)

model = Pipeline([
    ('features', FeatureUnion([
        ('numeric_features', Pipeline([
            ('selector', get_numeric_data)
        ]))
    ])),
    ('clf', LinearSVC(class_weight='balanced', random_state=SEED))
])

In [18]:
def cross_val(X, y, model, kf):
    y = np.array(y)
    X = np.array(X)
    
    dicts = []
    
    for train_index, test_index in tqdm(kf.split(X, y)):

        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]
        X_train = pd.DataFrame(data=X_train[0:,0:],
                               columns=[feature_columns])
        X_test = pd.DataFrame(data=X_test[0:,0:],
                               columns=[feature_columns])

        X_train[feature_columns] = X_train[feature_columns].apply(pd.to_numeric)
        X_test[feature_columns] = X_test[feature_columns].apply(pd.to_numeric)
        
        
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)
        
        report = classification_report(y_test, y_pred, output_dict=True)
        dicts.append(report)
    
    return dicts

In [19]:
kf = StratifiedKFold(n_splits=10, shuffle=True, random_state=SEED)
dicts = cross_val(X, y, model, kf)

In [20]:
classes = ['best', 'worst', 'average', 
           'micro avg', 'macro avg', 'weighted avg']

In [21]:
result = {
    'precision': [],
    'recall': [], 
    'f1-score': []
}

for cl in classes:
    precision = []
    recall = []
    f1_score = []
    support = []
    for d in dicts:
        precision.append(d[cl]['precision'])
        recall.append(d[cl]['recall'])
        f1_score.append(d[cl]['f1-score'])
    result['precision'].append(round(statistics.mean(precision), 3))
    result['recall'].append(round(statistics.mean(recall), 3))
    result['f1-score'].append(round(statistics.mean(f1_score),3))
result = pd.DataFrame(data=result, index=classes)
result

,precision,recall,f1-score
best,0.263,0.213,0.123
worst,0.506,0.548,0.416
average,0.699,0.498,0.477
micro avg,0.480,0.480,0.480
macro avg,0.489,0.420,0.339
weighted avg,0.596,0.480,0.420


# Feature importance

In [22]:
from sklearn.ensemble import RandomForestClassifier
from boruta import BorutaPy

In [23]:
y = np.array(y)
X = np.array(X)
y = y.ravel()

In [25]:
rf = RandomForestClassifier(n_jobs=-1, class_weight='balanced', max_depth=5)
feat_selector = BorutaPy(rf, n_estimators='auto', verbose=0, random_state=SEED)
feat_selector.fit(X, y)

BorutaPy(alpha=0.05,
     estimator=RandomForestClassifier(bootstrap=True, class_weight='balanced',
            criterion='gini', max_depth=5, max_features='auto',
            max_leaf_nodes=None, min_impurity_decrease=0.0,
            min_impurity_split=None, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            n_estimators=187, n_jobs=-1, oob_score=False,
            random_state=<mtrand.RandomState object at 0x10584e048>,
            verbose=0, warm_start=False),
     max_iter=100, n_estimators='auto', perc=100,
     random_state=<mtrand.RandomState object at 0x10584e048>,
     two_step=True, verbose=0)

In [33]:
for i, feature in enumerate(feat_selector.support_):
    if feature == True:
        print(feature_columns[i])

AdvV
CTTR
CVVI
D
LFP_first_procent
LFP_none
LFP_third_procent
LS
LV
LogTTR
MCI
ModV
NDW
NV
RTTR
SVVI
TTR
VVII
acl
advcl
av_depth
count_sent
count_tokens
density
der_suff3
der_suff4
der_suff6
freq_finite_forms
gerund_tokens
infinitive_tokens
link_Clarification
link_Contrast
link_Sentence connectors(i.e.)
link_Subordinators
link_Subordinators(while)
link_all
max_depth
mean_len_sent
num_4grams
parts
pres_sg_tokens
tokens_before_root
